In [36]:
import pandas as pd

In [37]:
data = pd.read_csv("countyall.csv")

In [38]:
def categorize_hotspot(cases):
    if cases >= 50:
        return True
    else:
        return False

data['Hotspot'] = data['weeklyper100k'].apply(categorize_hotspot)

In [39]:
data.sort_values(by=['county', 'date'], inplace=True)

# Shift the 'Hotspot' column by 14 days to create a new column indicating hotspot status in two weeks
data['Hotspot_in_2_weeks'] = data.groupby('county')['Hotspot'].shift(-14)
data

,Unnamed: 0,date,county,state,fips,cases,deaths,Population,casesprop,deathsprop,dailycase,weeklycase,dailydeath,weeklydeath,weeklyper100k,tranlevel,Minority_pct,region,Hotspot,Hotspot_in_2_weeks
0,0,2021-07-01,Alamance,North Carolina,37001.0,19035,289.0,175029.0,0.1088,0.0017,0.0,0.0,0.0,0.0,0.00,Low,0.3825,Northern,False,False
100,100,2021-07-02,Alamance,North Carolina,37001.0,19042,289.0,175029.0,0.1088,0.0017,7.0,0.0,0.0,0.0,0.00,Low,0.3825,Northern,False,False
200,200,2021-07-03,Alamance,North Carolina,37001.0,19042,289.0,175029.0,0.1088,0.0017,0.0,0.0,0.0,0.0,0.00,Low,0.3825,Northern,False,False
300,300,2021-07-04,Alamance,North Carolina,37001.0,19042,289.0,175029.0,0.1088,0.0017,0.0,0.0,0.0,0.0,0.00,Low,0.3825,Northern,False,False
400,400,2021-07-05,Alamance,North Carolina,37001.0,19042,289.0,175029.0,0.1088,0.0017,0.0,0.0,0.0,0.0,0.00,Low,0.3825,Northern,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15062,15099,2021-11-28,Yancey,North Carolina,37199.0,2996,35.0,18320.0,0.1635,0.0019,0.0,64.0,0.0,0.0,349.34,High,0.0868,Western,True,NaN
15162,15199,2021-11-29,Yancey,North Carolina,37199.0,3020,35.0,18320.0,0.1648,0.0019,24.0,45.0,0.0,0.0,245.63,High,0.0868,Western,True,NaN
15262,15299,2021-11-30,Yancey,North Carolina,37199.0,3024,35.0,18320.0,0.1651,0.0019,4.0,37.0,0.0,0.0,201.97,High,0.0868,Western,True,NaN
15361,15399,2021-12-01,Yancey,North Carolina,37199.0,3030,35.0,18320.0,0.1654,0.0019,6.0,34.0,0.0,0.0,185.59,High,0.0868,Western,True,NaN


In [40]:
# choosing categories for analysis

data_analysis = data[["date", "county", "Population", "cases", "Minority_pct", "Hotspot", "Hotspot_in_2_weeks"]]
data_analysis.to_csv("analysis_data.csv")

<bound method NDFrame.to_csv of           county  Population  cases  Minority_pct  Hotspot
0       Alamance    175029.0  19035        0.3825    False
1      Alexander     36172.0   4456        0.1416    False
2      Alleghany     11057.0   1095        0.1234    False
3          Anson     22028.0   2637        0.5152    False
4           Ashe     26506.0   2248        0.0806    False
...          ...         ...    ...           ...      ...
15456      Wayne    115895.0  17700        0.4677     True
15457     Wilkes     65151.0  11172        0.1344     True
15458     Wilson     79078.0  13595        0.5260     True
15459     Yadkin     36925.0   6399        0.1712     True
15460     Yancey     18320.0   3050        0.0868     True

[15461 rows x 5 columns]>